In [3]:
import pandas as pd
import re
from datetime import datetime

file_path = 'kodewilayah.txt'  # Ganti dengan path file txt Anda
data = pd.read_csv(file_path, header=None, names=['Kode', 'Nama'], sep=',')

# Fungsi untuk menghitung umur berdasarkan tanggal lahir
def hitung_umur(tanggal_lahir):
    today = datetime.today()
    tanggal_lahir = datetime.strptime(tanggal_lahir, "%d-%m-%Y")
    umur = today.year - tanggal_lahir.year - ((today.month, today.day) < (tanggal_lahir.month, tanggal_lahir.day))
    return umur

# Fungsi untuk mengidentifikasi wilayah berdasarkan kode NIK
def identifikasi_wilayah(kode_provinsi, kode_kota, kode_kecamatan, df_wilayah):
    provinsi_row = df_wilayah[df_wilayah['Kode'] == kode_provinsi]
    kota_row = df_wilayah[df_wilayah['Kode'] == kode_kota]
    kecamatan_row = df_wilayah[df_wilayah['Kode'] == kode_kecamatan]

    nama_provinsi = provinsi_row['Nama'].values[0] if not provinsi_row.empty else "Format NIK tidak valid."
    nama_kota = kota_row['Nama'].values[0] if not kota_row.empty else "Format NIK tidak valid."
    nama_kecamatan = kecamatan_row['Nama'].values[0] if not kecamatan_row.empty else "Format NIK tidak valid."

    return nama_provinsi, nama_kota, nama_kecamatan

# Fungsi utama untuk mengidentifikasi dan menghasilkan output NIK
def identifikasi_nik(nik, df_wilayah):
    # Regex untuk memecah bagian-bagian NIK
    pattern = r'(?P<kode_provinsi>\d{2})(?P<kode_kota>\d{2})(?P<kode_kecamatan>\d{2})(?P<tanggal>\d{2})(?P<bulan>\d{2})(?P<tahun>\d{2})\d{4}'
    match = re.match(pattern, nik)

    if match:
        kode_provinsi = match.group('kode_provinsi')
        kode_kota = f"{kode_provinsi}.{match.group('kode_kota')}"
        kode_kecamatan = f"{kode_kota}.{match.group('kode_kecamatan')}"

        # Validasi kode provinsi
        provinsi_row = df_wilayah[df_wilayah['Kode'] == kode_provinsi]
        if provinsi_row.empty:
            print(f"Format NIK tidak valid. \nKode provinsi {kode_provinsi} tidak ditemukan dalam data.")
            return

        # Validasi bulan
        bulan = match.group('bulan')
        if not (1 <= int(bulan) <= 12):
            print(f"Format NIK tidak valid.\nBulan {bulan} tidak ada.")
            return

        # Identifikasi wilayah (provinsi, kota/kabupaten, kecamatan)
        nama_provinsi, nama_kota, nama_kecamatan = identifikasi_wilayah(kode_provinsi, kode_kota, kode_kecamatan, df_wilayah)

        # Identifikasi jenis kelamin dan tanggal lahir
        tanggal = int(match.group('tanggal'))
        tahun = int(match.group('tahun'))

        # Identifikasi jenis kelamin
        if tanggal > 40:
            jenis_kelamin = 'Perempuan'
            tanggal -= 40
        else:
            jenis_kelamin = 'Laki-laki'

        # Imputasi tahun lahir (asumsi jika tahun lahir di bawah 25 maka 2000-an, jika tidak 1900-an)
        tahun_lahir = tahun + 2000 if tahun <= 25 else tahun + 1900

        # Format tanggal lahir
        tanggal_lahir = f"{tanggal:02d}-{bulan}-{tahun_lahir}"

        # Menghitung umur berdasarkan tanggal lahir
        umur = hitung_umur(tanggal_lahir)

        # Output hasil
        print(f"NIK: {nik}")
        print(f"Asal Provinsi: {nama_provinsi}")
        print(f"Asal Kota/Kabupaten: {nama_kota}")
        print(f"Asal Kecamatan: {nama_kecamatan}")
        print(f"Tanggal Lahir: {tanggal_lahir}")
        print(f"Usia: {umur}")
        print(f"Jenis Kelamin: {jenis_kelamin}")
    else:
        print("Format NIK tidak valid.")

,Kode,Nama
0,11,ACEH
1,11.01,KAB. ACEH SELATAN
2,11.01.01,Bakongan
3,11.01.01.2001,Keude Bakongan
4,11.01.01.2002,Ujong Mangki
...,...,...
91215,92.71.10,Maladum Mes
91216,92.71.10.1001,Suprau
91217,92.71.10.1002,Tampa Garam
91218,92.71.10.1003,Saoka
